<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [2]:
# Antes de empezar importamos las librerías necesarias
# Para tratamiento de datos
import numpy as np
import pandas as pd
# Para generar todas las combinaciones posibles
import itertools
# Para guardar DataFrames en Excel
from pandas import ExcelWriter
# Para gestión de fechas
from datetime import datetime

# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 


In [3]:
# Cargamos el archibo pkl generado en laboratorios anteriores
## Cargamos el archivo
df_catalogo = pd.read_pickle("datos/catalogo_completo.pkl")
df_catalogo.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,yes


## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

In [4]:
# Vamos a comprobar el formato de las columnas de nuestro archivo
df_catalogo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8807 non-null   object        
 1   type          8807 non-null   object        
 2   title         8807 non-null   object        
 3   director      6173 non-null   object        
 4   cast          7982 non-null   object        
 5   country       7976 non-null   object        
 6   date_added    8797 non-null   datetime64[ns]
 7   release_year  8807 non-null   int64         
 8   rating        8803 non-null   object        
 9   duration      3994 non-null   object        
 10  listed_in     8807 non-null   object        
 11  description   8807 non-null   object        
 12  Genre         513 non-null    object        
 13  Premiere      513 non-null    datetime64[ns]
 14  Runtime       513 non-null    float64       
 15  IMDB Score    513 non-null    float64      

In [5]:
# Podemos ver que la columna date_added ya tiene formato datetime
# El código necesario sería este: df_catalogo["date_added"] = pd.to_datetime(df_catalogo["date_added"], format="mixed")

In [6]:
# Vamos a generar la columna 'duration_cleaned' separando el valor numérico del texto
df_catalogo["duration_cleaned"] = df_catalogo["duration"].str.split(pat=" ", n=1, expand=True)[0]
# Creamos la columna aplicando el método str.plit a la columna "duration" indicando que el separador va a ser el espacio (pat=" ")
# n=1 indica que solo queremos que haga una división
# expand=True para señalar que queremos que separe cada elemento en columnas distintas
# [0] para que solo recoja en la columna nueva el primer elemento de la lista generada, en este caso sería el número
df_catalogo.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,yes,90
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaT,NaN,NaN,NaN,NaN,2


#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [7]:
# Vamos a ver los valores únicos de la columna 'rating'
df_catalogo["rating"].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

Las categorías serán las siguientes:
    - 'General Audience' = "PG", "TV-PG", "TV-G", "G", "TV-Y", "TV-Y7", "TV-Y7-FV"
    - 'Teens' = "PG-13", "TV-14"
    - 'Adults' = "TV-MA", "R", "NR", "UR", "NC-17"
Ignoramos los valores nulos "nan" y los valores en minutos porque corresponden a otra columna

In [8]:
#definimos la función que vamos a aplicar
def clasificacion_edades(categoria):
    """clasifica en grupos de edad según la categoría

    Args:
        categoria (str)): categoría de la serie o película a clasificar

    Returns:
        str: _grupo de edad que corresponde a la categoría
    """
    # Generamos un diccionario que contiene las categorías que corresponden a cada grupo de edad
    dict_categoria = {"General Audience": ["PG", "TV-PG", "TV-G", "G", "TV-Y", "TV-Y7", "TV-Y7-FV"], "Teens": ["PG-13", "TV-14"], "Adults": ["TV-MA", "R", "NR", "UR", "NC-17"]}
    # bucle que compruebe si la categoría está en el el diccionario y nos devuelve la clave correspondiente
    for clave, valor in dict_categoria.items():
        if categoria in valor:
            return clave

In [9]:
df_catalogo["rating_age"] = df_catalogo["rating"].apply(clasificacion_edades)
df_catalogo.sample()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age
3655,s3656,TV Show,Typewriter,NaN,"Aarna Sharma, Aaryansh Malviya, Mikail Gandhi,...",India,2019-07-19,2019,TV-MA,NaN,"International TV Shows, TV Horror, TV Mysteries",Three young friends in Goa plan to search an o...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults


#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [ ]:
# Creamos lista de actores famosos
actores_famosos =["Leonardo DiCaprio", "Tom Hanks", "Morgan Freeman"]
# aplicamos una lambda sobre la columna "cast" (x) que devuelve True si cualquier actor de x aparece en la lista de actores que hemos definido
df_catalogo["has_famous_actor"] = df_catalogo["cast"].apply(lambda x : True if any(actor in str(x) for actor in actores_famosos) else False)
df_catalogo[["cast", "has_famous_actor"]].sample(5)


,cast,has_famous_actor
3931,Bear Grylls,False
2518,"Lee Byung-hun, Kim Tae-hee, Jeong Jun-ho, Kim ...",False
2361,"Khaled El Nabawy, Hanan Turk, Yousra, Mahmoud ...",False
4073,"Bea Alonzo, Toni Gonzaga, Angel Locsin, Shaina...",False
3285,Fadily Camara,False


#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [11]:
df_catalogo["is_recent"] = df_catalogo["release_year"].apply(lambda x : True if x>= (pd.Timestamp.now().year -5) else False)
# creamos una nueva columna aplicando una función lambda sobre la columna "release_year" del dataframe df_catalogo, que será x en la lambda
# si x es mayor o igual al año actual - 5, es decir, en los últimos 5 años, la lambda devuelve True, en otro caso devuelve False
df_catalogo.sample(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent
921,s922,TV Show,StartUp,NaN,"Adam Brody, Edi Gathegi, Otmara Marrero, Marti...",United States,2021-05-04,2018,TV-MA,NaN,"Crime TV Shows, TV Dramas",An attempt to launder stolen money finances a ...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False
4689,s4690,TV Show,The Innocents,NaN,"Sorcha Groundsell, Percelle Ascott, Guy Pearce...",United Kingdom,2018-08-24,2018,TV-MA,NaN,"British TV Shows, International TV Shows, Roma...",Runaway teen lovers June and Harry find themse...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False
7575,s7576,Movie,Nazi Concentration Camps,George Stevens,NaN,United States,2017-03-31,1945,TV-MA,59 min,"Classic Movies, Documentaries",Shocking footage shows Nazi concentration camp...,NaN,NaT,NaN,NaN,NaN,NaN,59,Adults,False,False


#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [12]:
# vamos a ver los valores máximo y mínimo de la columna "release_year"
df_catalogo["release_year"].max()
print(f"El valor mínimo de la columna 'release_year' es {df_catalogo["release_year"].min()} y el máximo es {df_catalogo["release_year"].max()}")

El valor mínimo de la columna 'release_year' es 1925 y el máximo es 2021


In [13]:
# usaremos el método pd.cut para agrupar los años en sus décadas
# definimos las listas de bins y labels que luego usaremos en el método
years_bins = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030]
decadas = ["1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]
# marcamos que se incluya el extremo izquierdo de cada intervalo 'right = False'

In [14]:
df_catalogo["decade"] = pd.cut(df_catalogo["release_year"], bins = years_bins, right = False, labels = decadas)
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade
1573,s1574,TV Show,The Expanding Universe of Ashley Garcia,NaN,"Paulina Chávez, Jencarlos Canela",United States,2020-12-09,2020,TV-PG,NaN,"Kids' TV, TV Comedies, Teen TV Shows",15-year-old scientist Ashley Garcia explores t...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,General Audience,False,True,2020s
722,s723,Movie,Sir! No Sir!,David Zeiger,Troy Garity,United States,2021-06-15,2005,TV-MA,84 min,Documentaries,This documentary chronicles the largely forgot...,NaN,NaT,NaN,NaN,NaN,NaN,84,Adults,False,False,2000s


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [15]:
# En ambos casos usamos el método .str.split. En los dos marcamos que queremos una columna nueva (expand = True) con el primer elemento obtenido poniendo el [0] al final
# Para extraer el primer actor usamos como separador la coma, ya que los nombres del cast está separados por comas
df_catalogo["first_actor"] = df_catalogo["cast"].str.split(pat=",", n=-1, expand=True)[0]
# Para extraer el primer nombre del director usamos el espacio como separador, para que extraiga solo la primera palabra
df_catalogo["first_name_director"] = df_catalogo["director"].str.split(pat=" ", n=-1, expand=True)[0]
df_catalogo.sample(2)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director
5619,s5620,Movie,Heroes Wanted,Joaquín Mazón,"María León, Miki Esparbé, Jordi Sánchez, Andon...",Spain,2017-02-01,2016,TV-MA,NaN,"Action & Adventure, Comedies, International Mo...","When its special ops agency is wiped out, Spai...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False,2010s,María León,Joaquín
4538,s4539,Movie,Truth or Dare,Nick Simon,"Cassandra Scerbo, Brytni Sarpy, Mason Dye, Har...",United States,2018-10-03,2017,TV-MA,NaN,Horror Movies,"A game of ""Truth or Dare"" among college friend...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False,2010s,Cassandra Scerbo,Nick


#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).


In [16]:
# Usamos el método fillna() para rellenar los valores nulos con un valor específico, en este caso "sin información"
df_catalogo["cast"] = df_catalogo["cast"].fillna("sin información")
# Aplicamos el método str.strip() para eliminar posibles espacios en blanco
df_catalogo["cast"] = df_catalogo["cast"].str.strip()
# aplicamos una lambda que nos de la longitud de la lista generada al separar los elementos del cast en cada fila
df_catalogo["num_cast"] = df_catalogo["cast"].apply(lambda x: len(x.split(",")))
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast
1807,s1808,Movie,Exes Baggage,Dan Villegas,"Angelica Panganiban, Carlo Aquino, Dionne Mons...",Philippines,2020-10-22,2018,TV-MA,NaN,"Dramas, International Movies, Romantic Movies","After years apart, a former couple reunites an...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False,2010s,Angelica Panganiban,Dan,3
6409,s6410,Movie,Camera Store,Scott Marshall Smith,"John Larroquette, John Rhys-Davies, Paul Ben-V...",United States,2017-08-20,2016,TV-MA,104 min,"Comedies, Dramas",With the digital revolution just around the co...,NaN,NaT,NaN,NaN,NaN,NaN,104,Adults,False,False,2010s,John Larroquette,Scott,8



#### Ejercicio 9: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.

In [17]:
# Usamos el método fillna() para rellenar los valores nulos con un valor específico, en este caso "sin información"
df_catalogo["director"] = df_catalogo["director"].fillna("sin información")
df_catalogo.sample(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast
4480,s4481,Movie,Ronnie Coleman: The King,Vlad Yudin,Ronnie Coleman,United States,2018-10-25,2018,TV-MA,NaN,Documentaries,"Today, legendary bodybuilder Ronnie Coleman's ...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,Adults,False,False,2010s,Ronnie Coleman,Vlad,1
4956,s4957,Movie,Phir Bhi Dil Hai Hindustani,Aziz Mirza,"Shah Rukh Khan, Juhi Chawla, Paresh Rawal, Sat...",India,2018-04-01,2000,TV-14,159 min,"Comedies, Dramas, International Movies","In this Bollywood entertainment, two journalis...",NaN,NaT,NaN,NaN,NaN,NaN,159,Teens,False,False,2000s,Shah Rukh Khan,Aziz,9


In [18]:
# Agrupamos por los directores obtenidos en el ejercicio 6 y usamos size para que cuente las apariciones
apariciones_director = df_catalogo.groupby("first_name_director").size().reset_index(name= "apariciones")
# usamos reset_index(name= "apariciones") para que aparezca una segunda columna llamada apariciones en el nuevo dataframe
apariciones_director

,first_name_director,apariciones
0,A.,4
1,A.R.,2
2,Aadish,1
3,Aamir,2
4,Aanand,1
...,...,...
2303,Çagan,1
2304,Ísold,1
2305,Óskar,1
2306,Ömer,2


In [19]:
# filtramos los nombres de directores cuyo número de apariciones es mayor a 1
directores_recurrentes = apariciones_director[apariciones_director["apariciones"] > 1]["first_name_director"]
# convertimos la serie generada a una lista usando el método tolist()
lista_directores_recurrentes = directores_recurrentes.tolist()

In [20]:
# filtramos si los nombres de la columna "first_name_director" están en la lista de directores recurrentes
df_catalogo["recurrent_director"] = df_catalogo["first_name_director"].isin(lista_directores_recurrentes)
# como el resultado es de tipo bool, aplicaremos un map para cambiarlo a "Yes" o "No"
mapa_recurrentes = {True : "Yes", False: "No"}
df_catalogo["recurrent_director"] = df_catalogo["recurrent_director"].map(mapa_recurrentes)
df_catalogo.sample(5)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original,duration_cleaned,rating_age,has_famous_actor,is_recent,decade,first_actor,first_name_director,num_cast,recurrent_director
6873,s6874,TV Show,Gonul,Neslihan Yesilyurt,"Gökçe Bahadır, Engin Öztürk, Onur Saylak",Turkey,2017-07-12,2015,TV-14,NaN,"International TV Shows, TV Dramas, TV Thrillers",A nurse escapes from her surgeon fiancé after ...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens,False,False,2010s,Gökçe Bahadır,Neslihan,3,No
2188,s2189,Movie,Sugar High,Ariel Boles,Hunter March,United States,2020-07-31,2020,TV-G,44 min,Children & Family Movies,"Talented sugar artists compete for $10,000 ove...",NaN,NaT,NaN,NaN,NaN,NaN,44,General Audience,False,True,2020s,Hunter March,Ariel,1,Yes
5215,s5216,TV Show,In Laws,sin información,"Kelvin Leong, Remon Lim, Loo Aye Keng, Lawrenc...",NaN,2017-10-16,2016,TV-PG,NaN,"International TV Shows, Romantic TV Shows, TV ...","Once domineered by her own mother, a well-mean...",NaN,NaT,NaN,NaN,NaN,NaN,NaN,General Audience,False,False,2010s,Kelvin Leong,NaN,7,No
569,s570,Movie,Dreamy Eyes,Victor Vu,"Tran Nghia, Truc Anh, Tran Phong, Khanh Van, N...",Vietnam,2021-07-01,2019,TV-14,117 min,"Dramas, International Movies, Romantic Movies","Through heartbreak and betrayal, Ngan's unrequ...",NaN,NaT,NaN,NaN,NaN,NaN,117,Teens,False,True,2010s,Tran Nghia,Victor,12,Yes
5653,s5654,TV Show,My Runway,sin información,"Ji-yeon Park, Dong-ho Kang, Chul-woong Kang, B...",South Korea,2016-12-30,2016,TV-14,NaN,"International TV Shows, Korean TV Shows, Roman...",An egotistic top male model and a pretty model...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Teens,False,False,2010s,Ji-yeon Park,NaN,7,No
